In [ ]:
import src.models.helpers as mh
import src.models.builders as mb
import src.inference.helpers as ih
import src.data.breathe_data as breathe_data

import pandas as pd

In [ ]:
df = breathe_data.load_from_excel()
df.head()

In [ ]:
# Infer AR and IA for all data points given an individuals' age, sex, height, FEV1 and O2 saturation measurements
def infer_AR_IA_for_ID(df):
    df.reset_index(inplace=True)
    _, inf_alg, _, ecFEV1, AR, _, _, IA, _, O2Sat = mb.o2sat_fev1_point_in_time_model(df.Height[0], df.Age[0], df.Sex[0])
    # _, inf_alg, AR, IA, ecFEV1, O2Sat = build_model(173, 53, "Male")

    # inf_res = ih.infer(inf_alg, [AR, IA], [[ecFEV1, 1.31], [O2Sat, 97.0]], show_progress=False, joint=False, get_time=True)

    def infer_and_unpack(ecfev1_obs, o2sat_obs):
        res = ih.infer(
            inf_alg,
            [AR, IA],
            [[ecFEV1, ecfev1_obs], [O2Sat, o2sat_obs]],
            show_progress=False,
            joint=False,
            get_time=False,
        )
        # mean_ar = AR.get_mean(res[AR.name].values)
        # mean_ia = IA.get_mean(res[IA.name].values)
        # return mean_ar, mean_ia
        return res[AR.name].values, res[IA.name].values

    res = df.apply(
        lambda row: infer_and_unpack(row["ecFEV1"], row["O2 Saturation"]),
        axis=1,
    )
    return res


resraw = df.groupby("ID").apply(infer_AR_IA_for_ID)
# resraw = df.iloc[np.r_[10:13, 1000:1007]].groupby("ID").apply(infer_AR_IA_for_ID)
res = (
    resraw.apply(pd.Series)
    .reset_index()
    .rename(columns={0: "AR", 1: "IA"})
    .drop(columns="level_1")
)
res

In [ ]:
AR = mh.variableNode("Airway resistance (%)", 0, 90, 2, prior={"type": "uniform"})
IA = mh.variableNode("Inactive alveoli (%)", 0, 30, 1, prior={"type": "uniform"})

res["AR mean"] = res["AR"].apply(lambda x: AR.get_mean(x))
res["IA mean"] = res["IA"].apply(lambda x: IA.get_mean(x))

In [ ]:
res

In [ ]:
# Save rawres as excel to ../../../../ExcelFiles
res.to_excel(
    "../../../../ExcelFiles/inferred_AR_IA_with_FEV1_O2Sat_no_AR-IA_factor.xlsx"
)

# Save arrays as objects
Arrays are currently saved as strings which is painful to process when reading from the file

In [ ]:
import pandas as pd
import numpy as np
import src.data.helpers as data_helpers

In [ ]:
data_helpers.load_excel("../../../../ExcelFiles/inferred_AR_IA_with_FEV1_O2Sat_no_AR-IA_factor_1.xlsx", ["AR", "IA"]).iloc[0].AR

In [ ]:
df = pd.read_excel("../../../../ExcelFiles/inferred_AR_IA_with_FEV1_O2Sat_no_AR-IA_factor_1.xlsx")

In [ ]:
# Convert back to arrays
def str_to_array(s):
    s_cleaned = s.replace('\\n', '')
    return np.fromstring(s_cleaned[1:-1], sep=' ')

df["AR"] = df.AR.apply(str_to_array)
df["IA"] = df.IA.apply(str_to_array)

In [ ]:
df.iloc[0].AR

In [ ]:
df.to_excel(
    "../../../../ExcelFiles/inferred_AR_IA_with_FEV1_O2Sat_no_AR-IA_factor_1.xlsx", index=False
)

In [ ]:
df = pd.read_excel("../../../../ExcelFiles/inferred_AR_IA_with_FEV1_O2Sat_no_AR-IA_factor.xlsx")

In [ ]:
df.AR.apply(json.loads)[0]